In [ ]:
import csv
import re
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter
from string import punctuation

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
data_path = './../../../data'
advocate_dir = os.path.join(data_path, 'BeerAdvocate')
reviews_ba = pd.read_csv(os.path.join(advocate_dir, 'ratings_BA_clean.csv'))

rev_true_ba = reviews_ba[reviews_ba['review']==True]

FileNotFoundError: [Errno 2] No such file or directory: './../../../data/BeerAdvocate/ratings_BA_clean.csv'

In [ ]:
users = pd.read_csv(os.path.join(advocate_dir, 'users.csv'))
beer_BA = pd.read_csv(os.path.join(advocate_dir, 'beers.csv'))

In [ ]:
beer_BA_grouped = beer_BA.groupby('style')
most_fam_b_per_style = pd.DataFrame()
for name, gruop in beer_BA_grouped:
    g = gruop.sort_values(ascending=[False],by=['nbr_reviews'])
    most_fam_b_per_style = pd.concat([most_fam_b_per_style, g.head(1)])

most_fam_stout = most_fam_b_per_style.sort_values(ascending=[False],by=['nbr_reviews']).iloc[1]
reviews_Stout = reviews_ba[reviews_ba['beer_id']==most_fam_stout.beer_id]
text_rev_stout = reviews_Stout[reviews_Stout['review']==True]['text']

In [113]:
expert_terms = {
    "flavor": ['caramel', 'toffee', 'bready', 'biscuity', 'nutty', 'roasted', 'chocolate', 'coffee', 'mocha','molasses','syrupy','smokey','toasted','vanilla','cocoa','sugar',
              'hoppy', 'citrusy', 'tropical', 'floral', 'piney', 'herbal', 'grassy', 'earthy', 'dank', 'spicy', 'peppery', 'juicy', 'zesty', 'tangy', 'bitterness',
               'estery','fruity', 'berry',' cherry',' apple',' banana', 'pear', 'stone fruit', 'plum', 'fig', 'raisin', 'peach',' apricot', 'dark fruit', 'citrus zest', 'lemon', 'peel', 'orange','pineapple', 'mango', 'lychee',
               'oaky', 'woody', 'bourbon', 'whiskey', 'rye', 'tequila', 'brandy','vinous', 'tart', 'lacto', 'honey', 'clove', 'bubblegum', 'funky', 'currant', 'grape','tea','blueberry','blackberry','cassis',
               'gooseberry','loganberry','nectar','nectarine','pomegranate','quince','rasberry','strawberry','watermelon','grapefruit','lemongrass','lime','mandarin','tangerine','coconut','cream','guava','melon',
               'papaya','rose','cedar','cognac','hay','tobacco','mint','sage','thyme','aniseed','cinnamon','ginger','pepper','licorice','candy','cucumber','garlic',
               'bouquet', 'aromatic', 'fragrant','perfumed', 'faint', 'musky', 'subtle', 'yeasty', 'malty', 'pungent','burnt'],
    "mouthfeel": [ 'body','full-bodied', 'medium-bodied', 'light-bodied', 'unfolding','aftertaste','thick', 'thin', 'creamy', 'smooth', 'velvety', 'oily','tannic', 'chalky', 'drying', 'slick', 'watery', 'effervescent', 'fizzy', 'tingly', 'prickly', 'carbonated', 'viscosity', 'warming', 'coating', 'biting', 'sharp','rusty','evolving'],
    "brewing": ['wheat','dry','hopping', 'barrel', 'conditioned', 'fermentation', 'wort', 'mash', 'sparging', 'crashing', 'decoction', 'conditioning', 'strain', 'adjuncts', 'grains', 'malt'],
    "technical": [ 'balance', 'palate','complexity', 'depth', 'layers', 'nuanced', 'refined', 'structured', 'profile', 'round', 'harmonious', 'clean', 'crisp', 'finish', 'lingering', 'evolving', 'clarity', 'purity', 'dense', 'robust', 'powerful', 'restrained', 'vibrant', 'subdued', 'heavy', 'integrity', 'layered', 'benchmark', 'classic', 'signature', 'finesse', 'elegance',
                  'exemplar', 'well-integrated', 'classic example', 'flawless', 'exceptional', 'outstanding', 'world-class', 'traditional', 'innovative', 'unconventional', 'unique'],
    "appearance": ['hazy', 'cloudy', 'opaque', 'translucent', 'clear', 'bright', 'unfiltered', 'filtered', 'lacing', 'foam', 'frothy', 'rocky head', 'stable head', 'gold', 'amber', 'ruby', 'copper', 'dark', 'black', 'mahogany', 'light', 'straw', 'yellow', 'chestnut', 'tan', 'off-white','blonde','bronze','coal','ebony','inky','milky','muddy','obsidian','sepia', 'sunlight','transparent',],
    "off_flavors": [ 'rotten','sulphur','egg','goat','sour','tasteless','inferior','acidic','oxidized', 'piss','metallic', 'cardboard', 'stale', 'sulfur', 'skunky', 'diacetyl', 'astringent', 'phenolic', 'acetaldehyde', 'DMS', 'cloying', 'solvent-like', 'overly bitter', 'thin', 'harsh', 'vegetal', 'buttery', 'musty, moldy', 'medicinal' ],
}


#flavour_terms_synonyms = {'chocolaty':'chocolate', 'chocoholic':'chocolate', 'chocolate':'chocolate'}


In [ ]:
stemmer = nltk.stem.SnowballStemmer('english')
stemmer.stem('initial impression')

'initial impress'

In [114]:
expert_terms_stemmed = {}
for category, terms in expert_terms.items():
    tokens = [word_tokenize(term.lower()) for term in terms]

    #stemmer = PorterStemmer()
    #lemma = nltk.wordnet.WordNetLemmatizer()
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmed_tokens = [stemmer.stem(word[0]) for word in tokens]
    #lemmad_tokens = [lemma.lemmatize(word[0]) for word in tokens]
    print(stemmed_tokens)
    expert_terms_stemmed[category] =  stemmed_tokens



['caramel', 'toffe', 'breadi', 'biscuiti', 'nutti', 'roast', 'chocol', 'coffe', 'mocha', 'molass', 'syrupi', 'smokey', 'toast', 'vanilla', 'cocoa', 'sugar', 'hoppi', 'citrusi', 'tropic', 'floral', 'piney', 'herbal', 'grassi', 'earthi', 'dank', 'spici', 'pepperi', 'juici', 'zesti', 'tangi', 'bitter', 'esteri', 'fruiti', 'berri', 'cherri', 'appl', 'banana', 'pear', 'stone', 'plum', 'fig', 'raisin', 'peach', 'apricot', 'dark', 'citrus', 'lemon', 'peel', 'orang', 'pineappl', 'mango', 'lyche', 'oaki', 'woodi', 'bourbon', 'whiskey', 'rye', 'tequila', 'brandi', 'vinous', 'tart', 'lacto', 'honey', 'clove', 'bubblegum', 'funki', 'currant', 'grape', 'tea', 'blueberri', 'blackberri', 'cassi', 'gooseberri', 'loganberri', 'nectar', 'nectarin', 'pomegran', 'quinc', 'rasberri', 'strawberri', 'watermelon', 'grapefruit', 'lemongrass', 'lime', 'mandarin', 'tangerin', 'coconut', 'cream', 'guava', 'melon', 'papaya', 'rose', 'cedar', 'cognac', 'hay', 'tobacco', 'mint', 'sage', 'thyme', 'anise', 'cinnamon',

In [115]:
exp_stem_set = []
for category, terms in expert_terms_stemmed.items():
    exp_stem_set = exp_stem_set + terms

exp_stem_set = set(exp_stem_set)

In [ ]:
def extract_keywords(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    punkt_symbols = set(punctuation)
    removable_words= stop_words.union(punkt_symbols)

    filtered_tokens = [word for word in tokens if ((word not in removable_words) and word.isalpha())]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    return stemmed_tokens

In [ ]:
def exp_stem_extract(text, exp_stem_set):
    extracted_keywords = set(extract_keywords(text))
    return list(exp_stem_set.intersection(extracted_keywords))

In [ ]:
def stem_df(x, exp_stem_set):
    x = x.copy()
    x.loc[:, 'text_lower'] = x['text'].str.lower()
    stems =  x['text_lower'].apply(lambda x: exp_stem_extract(x,exp_stem_set))
    x = x.drop(columns=['text', 'text_lower'])
    return pd.concat([x, stems], axis=1).rename(columns={'text_lower':'stems'})

In [ ]:
[len(expert_terms[category]) for category in expert_terms.keys()]

[73, 19, 27, 17, 34, 28, 17, 20, 26]

In [ ]:
[len(expert_terms_stemmed[category]) for category in expert_terms_stemmed.keys()]
#pd.DataFrame.from_dict(expert_terms_stemmed)

[73, 19, 27, 17, 34, 28, 17, 20, 26]

In [ ]:
for category in expert_terms_stemmed.keys():
    for i in range(len(expert_terms_stemmed[category])):
        print(category,';',expert_terms[category][i], ':', expert_terms_stemmed[category][i])

In [ ]:
text_rev_stout = reviews_Stout[reviews_Stout['review']==True][['user_id', 'text']]


NameError: name 'reviews_Stout' is not defined

In [ ]:
import time

start_time = time.time()

# Concatenate the results back with the original DataFrame
text_rev_stout = stem_df(text_rev_stout, exp_stem_set)

print("--- %s seconds ---" % (time.time() - start_time))

--- 2.859100341796875 seconds ---


In [ ]:
text_rev_stout

,user_id,stems
3820003,mdaniel723.1205050,[coffe]
3820012,cmurphy7.1176491,"[coffe, bodi, vanilla, sharp, aftertast, finis..."
3820015,jprince103.1152897,"[coffe, chocol, overli, note]"
3820016,docdeens.1196859,[balanc]
3820026,openstoutlover.1183725,[]
...,...,...
3826851,jake_ramrod.719193,"[coffe, depth, slick, head, earthi, dark, hint..."
3826852,qwest_35.716659,"[coffe, bitter, mocha, complex, roast, dark, w..."
3826853,michaelnickerson.724553,"[coffe, bitter, head, smooth, initi, finish, d..."
3826854,jgargano.723897,"[balanc, initi, mouthfeel]"


In [ ]:
start_time = time.time()

rev_true_ba = stem_df(rev_true_ba, exp_stem_set)

print("--- %s seconds ---" % (time.time() - start_time))

--- 1778.5225338935852 seconds ---


In [ ]:
rev_true_ba.to_pickle(os.path.join(advocate_dir, 'reviews_with_exp_stems.pkl'))

In [ ]:
len(rev_true_ba)

NameError: name 'rev_true_ba' is not defined

In [ ]:
rev_true_ba[rev_true_ba['lang_tag']!='en']['stems']

119                             [crisp]
121        [head, crisp, creami, clean]
126                                  []
128                                  []
130                                  []
                       ...             
4812581                              []
4812590                         [hoppi]
4812600                              []
4812644                              []
4812670                           [rye]
Name: stems, Length: 126810, dtype: object

In [ ]:
data_path = './../../../data'
rb_dir = os.path.join(data_path, 'RateBeer')
reviews_rb = pd.read_csv(os.path.join(rb_dir, 'ratings_RB_clean.csv'))

rev_true_rb = reviews_rb[reviews_rb['review']==True]

In [ ]:
len(rev_true_rb)

7121601

In [ ]:
rev_true_rb = rev_true_rb[rev_true_rb['lang_tag']=='en']

In [ ]:
len(rev_true_rb)

6606948

In [ ]:
import time
start_time = time.time()

rev_true_rb = stem_df(rev_true_rb, exp_stem_set)

print("--- %s seconds ---" % (time.time() - start_time))

--- 2529.523526906967 seconds ---


In [ ]:
rev_true_rb.to_pickle(os.path.join(rb_dir, 'reviews_with_exp_stems.pkl'))

In [ ]:
len(rev_true_rb)